# Personal Assistant


In [1]:
!pip install -qU openai
!pip install -qU langchain
!pip install -qU huggingface_hub
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.0 MB/s eta 0:00:00


In [2]:
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI, OpenAIChat
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)
prefix_message = "You are a personal assistant who helps me, Vitalij, \
to manage my productivity and efficiency on a daily base."

template = """Question: {question}

Answer: """

question = "How do I make my day more productive?"
system_message_prompt = SystemMessagePromptTemplate.from_template(prefix_message)
#humange_message_prompt = HumanMessagePromptTemplate.from_template(template)

human_message_prompt = HumanMessagePromptTemplate(
        prompt = PromptTemplate(template=template, input_variables=["question"])
    )

chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#prefix_message = [{"role": "system", "content": "You are a personal assistant who helps me, Vitalij, \
#to manage my productivity and efficiency on a daily base."}]

chat_gpt35 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature = 0, max_tokens = 100)

llm_chain = LLMChain(
    prompt=chat_prompt_template,
    llm=chat_gpt35
)

print(llm_chain.run(question))

To make your day more productive, here are some tips:

1. Set clear goals: Start your day by setting specific and achievable goals. This will give you a sense of direction and purpose.

2. Prioritize tasks: Identify the most important tasks that need to be done and prioritize them accordingly. Focus on completing high-priority tasks first.

3. Break tasks into smaller steps: Large tasks can be overwhelming. Break them down into smaller, more manageable steps to make them easier to tackle.

4


## Adding memory

### Most basic memory
Keep in mind, this blows up the token count. Everytime you send a request, you pay more and more.

In [4]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm = chat_gpt35,
    verbose = True,
    memory = memory
)

In [5]:
conversation.predict(input=question)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How do I make my day more productive?
AI:

> Finished chain.


'There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider'

In [6]:
conversation.predict(input="Create a list of strategies.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How do I make my day more productive?
AI: There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider
Human: Create a list of strategies.
AI:

> Finished chain.


'Sure! Here is a list of strategies to make your day more productive:\n\n1. Set clear goals and priorities for the day.\n2. Break down larger tasks into smaller, more manageable ones.\n3. Eliminate distractions as much as possible.\n4. Find a quiet workspace where you can concentrate.\n5. Prioritize your tasks based on importance and urgency.\n6. Use time management techniques, such as the Pomodoro Technique, to stay focused.\n7. Take regular breaks to recharge and avoid'

In [7]:
print(conversation.memory.buffer)

Human: How do I make my day more productive?
AI: There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider
Human: Create a list of strategies.
AI: Sure! Here is a list of strategies to make your day more productive:

1. Set clear goals and priorities for the day.
2. Break down larger tasks into smaller, more manageable ones.
3. Eliminate distractions as much as possible.
4. Find a quiet workspace where you can concentrate.
5. Prioritize your tasks based on importance and urgency.
6. Use time manageme

### Basic Conversation with ConversationBufferWindowMemory

Define how much previous messages should be preserved. Older messages are dropped.

In [8]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

from langchain.chains import ConversationChain

In [9]:
# k = 2: Keep only the last 2 interactions in memory
window_memory = ConversationBufferWindowMemory(k=2)

In [10]:
conversation = ConversationChain(
    llm = chat_gpt35,
    verbose = True,
    memory = window_memory
)

In [11]:
conversation.predict(input=question)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How do I make my day more productive?
AI:

> Finished chain.


'There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider'

In [12]:
conversation.predict(input="Create a list of strategies.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How do I make my day more productive?
AI: There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider
Human: Create a list of strategies.
AI:

> Finished chain.


'Sure! Here is a list of strategies to make your day more productive:\n\n1. Set clear goals and priorities for the day.\n2. Break down larger tasks into smaller, more manageable ones.\n3. Eliminate distractions as much as possible.\n4. Find a quiet workspace where you can concentrate.\n5. Prioritize your tasks based on importance and urgency.\n6. Use time management techniques, such as the Pomodoro Technique, to stay focused.\n7. Take regular breaks to recharge and avoid'

In [13]:
conversation.predict(input="What is the easiest to start with?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How do I make my day more productive?
AI: There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider
Human: Create a list of strategies.
AI: Sure! Here is a list of strategies to make your day 

'The easiest strategy to start with would be setting clear goals and priorities for the day. This can be done by making a to-do list or using a planner to outline the tasks you need to accomplish. By having a clear plan in place, you can stay focused and avoid feeling overwhelmed by the number of tasks you have to complete.'

In [14]:
conversation.predict(input="Give me a tutorial on this?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Create a list of strategies.
AI: Sure! Here is a list of strategies to make your day more productive:

1. Set clear goals and priorities for the day.
2. Break down larger tasks into smaller, more manageable ones.
3. Eliminate distractions as much as possible.
4. Find a quiet workspace where you can concentrate.
5. Prioritize your tasks based on importance and urgency.
6. Use time management techniques, such as the Pomodoro Technique, to stay focused.
7. Take regular breaks to recharge and avoid
Human: What is the easiest to start with?
AI: The easiest strategy to start with would be setting clear goals and priorities for the day. This can be done by making a to-do list or using 

'Certainly! Here\'s a step-by-step tutorial on setting clear goals and priorities for the day:\n\n1. Start by taking a few minutes in the morning to reflect on what you need to accomplish for the day. Consider both your short-term and long-term goals.\n\n2. Write down your goals and tasks in a to-do list or a planner. Be specific and break down larger tasks into smaller, more manageable ones. For example, instead of writing "Work on project," you can break it down into'

In [15]:
print(conversation.memory.buffer)

[HumanMessage(content='How do I make my day more productive?', additional_kwargs={}, example=False), AIMessage(content='There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider', additional_kwargs={}, example=False), HumanMessage(content='Create a list of strategies.', additional_kwargs={}, example=False), AIMessage(content='Sure! Here is a list of strategies to make your day more productive:\n\n1. Set clear goals and priorities for the day.\n2. Break down larger tasks into smaller, more manageable 

### Basic Conversation with ConversationBufferWindowMemory

Creates a summary of every messages. Very helpful for documentation reasons.

In [16]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

In [17]:
# k = 2: Keep only the last 2 interactions in memory
summary_memory = ConversationSummaryBufferMemory(llm = OpenAIChat(), max_token_limit=40)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [18]:
conversation = ConversationChain(
    llm = chat_gpt35,
    verbose = True,
    memory = summary_memory
)

In [19]:
conversation.predict(input=question)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How do I make my day more productive?
AI:

> Finished chain.


'There are several strategies you can try to make your day more productive. First, start by setting clear goals and priorities for the day. This will help you stay focused and avoid getting overwhelmed by tasks. Additionally, try breaking down larger tasks into smaller, more manageable ones. This will make them feel less daunting and easier to tackle. Another helpful tip is to eliminate distractions as much as possible. This could mean turning off notifications on your phone or finding a quiet workspace where you can concentrate. Finally, consider'

In [20]:
conversation.predict(input="Create a list of strategies.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks how to make their day more productive. The AI suggests several strategies, such as setting clear goals and priorities, breaking down larger tasks, eliminating distractions, and considering
Human: Create a list of strategies.
AI:

> Finished chain.


'Sure! Here is a list of strategies to make your day more productive:\n\n1. Set clear goals and priorities: Start your day by identifying the most important tasks you need to accomplish. This will help you stay focused and avoid getting overwhelmed by smaller, less significant tasks.\n\n2. Break down larger tasks: If you have a big project or task ahead of you, break it down into smaller, more manageable steps. This will make it easier to tackle and give you a sense of progress as you complete'

In [21]:
conversation.predict(input="What is the easiest to start with?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks how to make their day more productive. The AI suggests several strategies, such as setting clear goals and priorities, breaking down larger tasks, eliminating distractions, and considering time management techniques. The AI also creates a list of strategies, including setting clear goals, breaking down larger tasks, prioritizing tasks, eliminating distractions, and using time management techniques.
Human: What is the easiest to start with?
AI:

> Finished chain.


'The easiest strategy to start with would be eliminating distractions. This can be as simple as turning off notifications on your phone or computer, closing unnecessary tabs or apps, and finding a quiet and comfortable workspace. By minimizing distractions, you can focus better on your tasks and increase your productivity.'

In [22]:
conversation.predict(input="Give me a tutorial on this?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks how to make their day more productive. The AI suggests several strategies, such as setting clear goals and priorities, breaking down larger tasks, eliminating distractions, and considering time management techniques. The AI also creates a list of strategies, including setting clear goals, breaking down larger tasks, prioritizing tasks, eliminating distractions, and using time management techniques. The easiest strategy to start with, according to the AI, is eliminating distractions. This can be achieved by turning off notifications, closing unnecessary tabs or apps, and finding a quiet workspace, which will help increase focus and productivity.
Human: Give me a t

"Certainly! Here's a step-by-step tutorial on how to eliminate distractions and increase your productivity:\n\n1. Start by turning off notifications on your devices. This includes notifications from social media, email, messaging apps, and any other apps that tend to distract you. By doing this, you'll be able to focus on your tasks without constant interruptions.\n\n2. Close unnecessary tabs or apps on your computer or smartphone. Having too many tabs or apps open can be overwhelming and make it difficult to concentrate. Close"

In [23]:
print(conversation.memory.moving_summary_buffer)

The human asks how to make their day more productive. The AI suggests several strategies, such as setting clear goals and priorities, breaking down larger tasks, eliminating distractions, and considering time management techniques. It creates a list of strategies, including setting clear goals, breaking down tasks, prioritizing, eliminating distractions, and using time management techniques. The easiest strategy is to start by eliminating distractions, which can be achieved by turning off notifications, closing unnecessary tabs or apps, and finding a quiet workspace. In response to the human's request for a tutorial, the AI provides a step-by-step guide on how to eliminate distractions, starting with turning off notifications and closing unnecessary tabs or apps.


### Basic Conversation with Conversation Knowledge Graph Memory


In [34]:
from langchain.chains.conversation.memory import ConversationKGMemory
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

from langchain.chains import ConversationChain

In [35]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

In [36]:
conversation_with_kg =ConversationChain(
    llm = chat_gpt35,
    verbose = True,
    prompt = prompt,
    memory = ConversationKGMemory(llm=chat_gpt35)
)

In [37]:
conversation_with_kg.predict(input=question)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: How do I make my day more productive?
AI:

> Finished chain.


'There are several strategies you can try to make your day more productive. One effective approach is to prioritize your tasks and create a to-do list. This will help you stay organized and focused on what needs to be done. Additionally, breaking down larger tasks into smaller, more manageable ones can make them feel less overwhelming and increase your productivity. Another helpful tip is to eliminate distractions, such as turning off notifications on your phone or computer, finding a quiet workspace, or using productivity apps that block certain websites or'

In [38]:
conversation_with_kg.predict(input="Create a list of strategies.")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Create a list of strategies.
AI:

> Finished chain.


'Sure! Here are some strategies that you can consider:\n\n1. Setting clear goals: Clearly define what you want to achieve and break it down into smaller, manageable tasks.\n2. Prioritizing tasks: Determine which tasks are most important and tackle them first.\n3. Time management: Use techniques like the Pomodoro Technique or Eisenhower Matrix to effectively manage your time.\n4. Delegating tasks: If possible, delegate tasks to others to lighten your workload and focus on more important tasks.\n5'

In [39]:
conversation_with_kg.predict(input="What is the easiest to start with?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: What is the easiest to start with?
AI:

> Finished chain.


"The easiest thing to start with would be learning a new language. It's a great way to expand your horizons and connect with people from different cultures. Plus, there are plenty of resources available online and in-person to help you get started."

In [40]:
conversation_with_kg.predict(input="Give me a tutorial on this?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Give me a tutorial on this?
AI:

> Finished chain.


'Sure! What specifically would you like a tutorial on?'

In [41]:
print(conversation_with_kg.memory.kg.get_triples())

[('Human', 'day more productive', 'make'), ('Human', 'a list of strategies', 'Create'), ('Human', 'the easiest to start with', 'What is'), ('Human', 'tutorial', 'wants')]


## Entity Memory

In [24]:
from langchain import OpenAI, ConversationChain
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [25]:
## The propmpt
ENTITY_MEMORY_CONVERSATION_TEMPLATE.template

'You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations a

In [26]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0,
             max_tokens = 256)

In [27]:

conversation = ConversationChain(
    llm=chat_gpt35,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=chat_gpt35)
)

In [28]:
conversation.predict(input="Hi I am Sam. My TV is broken but it is under warranty.")



> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

"Hello Sam! I'm sorry to hear that your TV is broken, but it's great that it's still under warranty. I'd be happy to help you with that. What seems to be the issue with your TV?"

In [29]:
conversation.predict(input="How can I get it fixed. The warranty number is A512453")



> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

"To get your TV fixed under warranty, you can follow these steps:\n\n1. Contact the manufacturer: Look for the contact information of the manufacturer of your TV. This information is usually available on their website or in the warranty documentation that came with your TV. Reach out to them and explain the issue you're experiencing with your TV.\n\n2. Provide warranty details: When contacting the manufacturer, make sure to provide them with the warranty number A512453. This will help them verify the warranty status of your"

In [30]:
conversation.memory.entity_cache

['A512453']

In [31]:
conversation.predict(input="Can you send the repair person call Dave to fix it?.")



> Entering new  chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to e

"I'm sorry, but as an AI language model, I don't have the capability to send a repair person or make phone calls. However, you can contact the manufacturer and request a repair service. They will be able to assist you further and provide information on how to schedule a repair for your TV."

In [32]:
conversation.memory.entity_cache

['Dave']

In [33]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'A512453': 'The warranty number A512453 is needed to verify the warranty '
            'status of the TV.',
 'Dave': 'Dave is a repair person who the human wants to send to fix their TV.',
 'Sam': "Sam's TV is broken but it is under warranty.",
 'TV': 'The TV is broken but it is under warranty.'}


# Fact extraction from article

In [20]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

vitalijs_new_article = "The Detective Work: Finding the Habits. Now, imagine you’re a detective. \
You’ve got a notebook, and you’re trying to catch the „habit“ thief. \
ou write down everything you do in a day and check them off if they’re good for you, \
like eating veggies, or circle them if they’re not so good, like eating too many sweets. \
Rule 1: A habit must be visible. \
Making a habit „visible“ can be done in two ways: Environment Design and Mental Visibility. \
Environment Design: Making the cues or triggers for the habit visible in your physical environment. \
Mental Visibility: Making the habit a clear and conscious goal in your mind. \
Making the habit visible in your mental landscape. \
Acting upon both increases the likelihood of forming a new habit. \
Most of your habit triggers are subconscious, ingrained in your routine. \
If you don’t know the habit trigger, you can’t intervene. \
This can be problematic with harmful or unproductive habits. Why?\
You can’t take steps to replace a negative habit with a positive one. \
You iterate the cycle of bad, repeated negative behavior. \
Your harmful habits affect your physical and mental health."
# build prompt template for simple question-answering
template = """Extract the key facts out of this text. Don't include opinions. Remove the detective metaphors. Give each fact a number and keep them short sentences. : \n\n {text_input}"""

fact_extraction_prompt = PromptTemplate(template=template, input_variables=["text_input"])
fact_extraction_chain = LLMChain(
    prompt = fact_extraction_prompt,
    llm=chat_gpt35
)

facts = fact_extraction_chain.run(vitalijs_new_article)

print(facts)

1. A habit must be visible.
2. Making a habit visible can be done through environment design and mental visibility.
3. Most habit triggers are subconscious and ingrained in routine.
4. Without knowing the habit trigger, it's difficult to replace negative habits.
5. Harmful habits have a negative impact on physical and mental health.


In [45]:
twitter_post_prompt = PromptTemplate(
    input_variables = ["facts"],
    template = "You are an Twitter Ghostwriter. \
    Take the following list of facts and write a short engaging Tweet. \
    Explain the facts to high school graduate. \
    Do not use any emojis in your output. \
    Don't leave out key info: \n\n {facts}"
)

In [46]:
twitter_post_chain = LLMChain(llm=chat_gpt35, prompt = twitter_post_prompt)

twitter_post = twitter_post_chain.run(facts)

print(twitter_post)
len(twitter_post)

📢 Want to kick that bad habit? 🚫🙅‍♀️ Here's the secret: make it visible! ✨✍️ By designing your environment and improving mental visibility, you can finally break free! 🌟🔑 Uncover those subconscious triggers and replace the negatives with positives! Your health will thank you! 🌿💪💙 #PositiveChange #HealthyMindHealthyBody


320

In [51]:
from langchain.chains import SimpleSequentialChain, SequentialChain

full_chain = SimpleSequentialChain(chains=[fact_extraction_chain, twitter_post_chain], verbose = True)

response = full_chain.run(vitalijs_new_article)



> Entering new  chain...
1. A habit must be visible in order to be addressed.
2. Making a habit visible can be done through environment design and mental visibility.
3. Most habit triggers are subconscious and ingrained in routine.
4. Unknown habit triggers can prevent intervention and replacement of negative habits.
5. Harmful habits can affect physical and mental health.
"🌟 Did you know that making your habits visible is key to addressing and changing them? 💡 Whether it's through environment design or mental visibility, it's crucial to identify those triggers that are often hidden in our routines. 🔄 Unconscious and ingrained, these habits can be a real challenge to detect and replace. 🚫 Beware! Ignoring unknown triggers may hinder intervention, impacting your physical and mental health. Time to shed light on those harmful habits! #HabitAwareness #PositiveChanges"

> Finished chain.


# API Chains

In [52]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate

In [53]:
chat_gpt35 = ChatOpenAI(model_name='gpt-3.5-turbo', max_tokens=100)

In [55]:
from langchain.chains.api import open_meteo_docs

chain_new = APIChain.from_llm_and_api_docs(chat_gpt35, open_meteo_docs.OPEN_METEO_DOCS, verbose=True)

In [58]:
chain_new.run("What is the temperature like right now in Regensburg, Germany in degrees Celcius?")



> Entering new  chain...
https://api.open-meteo.com/v1/forecast?latitude=49.0193&longitude=12.0975&current_weather=true&temperature_unit=celsius
{"latitude":49.02,"longitude":12.099998,"generationtime_ms":0.19693374633789062,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":346.0,"current_weather":{"temperature":23.8,"windspeed":19.7,"winddirection":230.0,"weathercode":3,"is_day":1,"time":"2023-07-05T10:00"}}

> Finished chain.


'The current temperature in Regensburg, Germany is 23.8 degrees Celsius.'

---